In [ ]:
import json
import geopandas as gpd
from shapely.geometry import MultiPoint
import random
import folium
import folium.plugins as plugins
from shapely.geometry import MultiPoint
import pandas as pd
from shapely.geometry import Point
from folium import TileLayer

In [ ]:
# Langkah 1: Baca file CSV menggunakan pandas
df = pd.read_csv('../data/koordinat_masjid.csv')

# Langkah 2: Konversi DataFrame ke GeoDataFrame
# Misalkan kolom latitude dan longitude di CSV bernama 'lat' dan 'long'
geometry = [Point(xy) for xy in zip(df['long'], df['lat'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)
gdf.crs = 'EPSG:4326'
# gdf['centroid_latitude'] = gdf['geometry'].centroid.y
# gdf['centroid_longitude'] = gdf['geometry'].centroid.x
# gdf['geometry'].centroid.to_list()
gdf

In [ ]:
df

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import random

# Titik koordinat Ka'bah
kabah_coords = [39.8262, 21.4225]  # [longitude, latitude]

# Contoh data masjid (ganti dengan data asli Anda)
# data = {
#     'nama': ['Masjid A', 'Masjid B', 'Masjid C'],
#     'lat': [21.4225, 21.4226, 21.4227],
#     'long': [39.8262, 39.8263, 39.8264]
# }
# gdf = pd.DataFrame(data)

# Function to generate a random color
def random_color():
    return '#{:06x}'.format(random.randint(0, 0xFFFFFF))

# Create a dictionary mapping each nama to a random color
unique_distrik = gdf['nama'].unique()
color_dict = {distrik: random_color() for distrik in unique_distrik}

# Create traces for each mosque
traces = []
lines = []

for _, row in gdf.iterrows():
    # Add scatter trace for each mosque
    traces.append(go.Scattergeo(
        lon=[row['long']],
        lat=[row['lat']],
        mode='markers+text',
        marker=dict(
            size=10,
            color=color_dict[row['nama']],
            symbol='circle'
        ),
        text=row['nama'],
        textposition='top center',
        name=row['nama']
    ))
    
    # Add line trace from each mosque to Ka'bah
    lines.append(go.Scattergeo(
        lon=[row['long'], kabah_coords[0]],
        lat=[row['lat'], kabah_coords[1]],
        mode='lines',
        line=dict(
            width=2,
            color='red',
            dash='dash'
        ),
        showlegend=False
    ))

# Create the map
fig = go.Figure()

# Add the traces to the figure
for trace in traces:
    fig.add_trace(trace)

for line in lines:
    fig.add_trace(line)

# Update the layout

# Update layout untuk mengatur ukuran canvas dan posisi legend
fig.update_layout(
    title='Contoh Peta Globe 3D',
    geo=dict(
        scope='world',
        projection_type='orthographic',
        showland=True
    ),
    height=1600,  # Tinggi dalam piksel
    width=1000,  # Lebar dalam piksel
    legend=dict(
        orientation='h',  # Mengatur orientasi legend secara horizontal
        yanchor='top',    # Posisi vertikal legend
        y=-0.2,           # Jarak legend dari plot (nilai negatif untuk posisi di bawah plot)
        xanchor='center', # Posisi horizontal legend
        x=0.5            # Pusat legend di tengah plot
    )
)

# fig.show()
# Save the figure to an HTML file
fig.write_html('../web/map-html/globe.html', auto_open=True)
